In [1]:
# import thư viện cần thiết
# Load in our libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [2]:
# đọc dữ liệu
import os
df = pd.read_csv("AmesHousing.csv")
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [3]:
#Get number of rows and columns
df.shape

(2930, 82)

#### Tiền xử lý dữ liệu

In [4]:
# Do thuật toán hồi quy chỉ làm việc với dữ liệu dạng số nên ta chuyển các biến có kiểu categorical thành biến “Dummy”
#Since Regression needs numerical features,convert categorical columns into dummy variables
df1= pd.get_dummies(df)
df1.head()

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_Abnorml,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,1,526301100,20,141.0,31770,6,5,1960,1960,112.0,...,0,0,0,1,0,0,0,0,1,0
1,2,526350040,20,80.0,11622,5,6,1961,1961,0.0,...,0,0,0,1,0,0,0,0,1,0
2,3,526351010,20,81.0,14267,6,6,1958,1958,108.0,...,0,0,0,1,0,0,0,0,1,0
3,4,526353030,20,93.0,11160,7,5,1968,1968,0.0,...,0,0,0,1,0,0,0,0,1,0
4,5,527105010,60,74.0,13830,5,5,1997,1998,0.0,...,0,0,0,1,0,0,0,0,1,0


In [5]:
# Kiểm tra và liệt kê những cột thiếu dữ liệu
#Look for columns with any NaN(missing) values
df1.columns[df1.isna().any()].tolist()

#Number of NaN values columnwise
df1.isna().sum()

Order                       0
PID                         0
MS SubClass                 0
Lot Frontage              490
Lot Area                    0
                         ... 
Sale Condition_AdjLand      0
Sale Condition_Alloca       0
Sale Condition_Family       0
Sale Condition_Normal       0
Sale Condition_Partial      0
Length: 307, dtype: int64

In [6]:
# Để xử lý dữ liệu bị thiếu ta thay bằng giá trị trung vị của cột tương ứng. Ta định nghĩa hàm impute_median() để thực hiện việc này
#Define function to impute series with it's median
def impute_median(series):
    return series.fillna(series.median())
df1['Lot Frontage']= df1['Lot Frontage'].transform(impute_median)
df1['Mas Vnr Area']=df1['Mas Vnr Area'].transform(impute_median)
df1['BsmtFin SF 1']=df1['BsmtFin SF 1'].transform(impute_median)
df1['BsmtFin SF 2']=df1['BsmtFin SF 2'].transform(impute_median)
df1['Bsmt Unf SF']=df1['Bsmt Unf SF'].transform(impute_median)
df1['Total Bsmt SF']=df1['Total Bsmt SF'].transform(impute_median)
df1['Bsmt Full Bath']=df1['Bsmt Full Bath'].transform(impute_median)
df1['Bsmt Half Bath']=df1['Bsmt Half Bath'].transform(impute_median)
df1['Garage Cars']=df1['Garage Cars'].transform(impute_median)
df1['Garage Area']=df1['Garage Area'].transform(impute_median)
#Check remaining columns with NaN values
df1.columns[df1.isna().any()].tolist()

['Garage Yr Blt']

In [7]:
# Sau khi thay trung vị vẫn còn trường (cột) ‘Garage Yr Blt’ thiếu dữ liệu. Ta sẽ lọai bỏ – không xét đến cột này, khi xây dựng mô hình đồng thời đưa dữ liệu đã tiền xử lý vào mảng df2 để tính toán trong các bước tiếp theo
#Drop this column
df2=df1.drop('Garage Yr Blt',axis=1)

#### Xử lý dữ liệu

In [8]:
# Đặt mảng y, chứa giá bán của các ngôi nhà, tương ứng cột SalePrice, chính là đầu ra của mô hình. Mảng X, chứa tất cả các cột còn lại (ngoại trừ cột SalePrice
#Define target array y
y= df2['SalePrice'].values

#Create feature array X
X= df2.drop('SalePrice',axis=1).values

In [9]:
#Check X's shape
X.shape

(2930, 305)

In [10]:
#Check Y's shape
y.shape

(2930,)

In [11]:
# Dùng lệnh reshape để định dạng y thành vector cột (mảng 1 cột) và kiểm tra
#Reshape y to have 1 column
y=y.reshape(-1,1)
y.shape

(2930, 1)

#### Chia dữ liệu

In [12]:
# Ta dùng hàm train_test_split() trong thư viện scikit-learn để chia bộ dữ liệu ta có thành 2 phần: dữ liệu huẩn luyện (train data) và dữ liệu kiểm tra (test data). Tham số test_size=0.3 cho biết, 30% dữ liệu được lấy để kiểm tra, 70% được dùng để huấn luyện mô hình
#Split the arrays into training and testing data sets
X_train, X_test,y_train, y_test= train_test_split(X,y,test_size=0.3,random_state=42)

#### Huấn luyện mô hình

In [13]:
# Đưa dữ liệu vào huấn luyện với mô hình hồi quy tuyến tính trong thư viện scikit-learn. Sau bước này, các tham số của mô hình đã được xác định
#Create a regressor object
LR= LinearRegression()

#Fit training set to the regressor
LR.fit(X_train,y_train)

#print("Mô hình hồi quy tuyến tính đã được huấn luyện, có các tham số:")
#print("Intercept =", LR.intercept_)
#print("Coefficients:", LR.coef_)

LinearRegression()

In [14]:
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

In [17]:
# Ta dùng mô hình tính toán giá nhà với các bộ dữ liệu kiểm tra. Kết quả tính toán được lưu trong mảng y_predIn
#Make predictions with the regressor
y_prediction = LR.predict(X_test)

In [19]:
#### Đánh giá mô hình
# Tính R2-score và Phương sai RMSE
# Calculate R2-score
score=r2_score(y_test,y_prediction)
print('R2-score is ',score)
print('Mean_sqrd_error is==',mean_squared_error(y_test,y_prediction))
print('Root_mean_squared error of is==',np.sqrt(mean_squared_error(y_test,y_prediction)))

R2-score is  0.8955304226952571
Mean_sqrd_error is== 734361363.6168106
Root_mean_squared error of is== 27099.10263489938
